# N0 Wavelength Study with PML

Having added exact pml, we compare getting N0 results without and with PML

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton

# Initialize

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
nu = 1

n_air = 1.00027717
n_glass = 1.4388164768221814

We'll make an instance for use with exact PML and one for without.

In [ ]:
d = 4.0775e-05
ts = [d, .5*d]
ns = [lambda x: n_air, lambda x: n_glass]
mats = ['air', 'glass']
maxhs = [.060, .010]
scale = 15e-6

A = BraggExact(ts=ts, maxhs=maxhs, ns=ns, mats=mats, scale=scale)

In [ ]:

ts = [4.0775e-05, 1e-5, 1.5e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]

mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [.06,.01, .02]

pml = {'alpha':.04, 'R0':rhos[-2]}

B = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs)

Just test them here

In [ ]:
k_low = A.ks[0].real * A.scale

plotlogf(A.determinant, .999*k_low, 1.001 * k_low, -.015,.015, nu, 'h2',
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(.99998*k_low)
imag = 0
x0 = np.array(guess + imag*1j)

beta = newton(A.determinant, x0, args=(nu, 'h2'), tol = 1e-17)
beta

In [ ]:
Fs = A.all_fields(beta, nu, 'h2')

In [ ]:
Draw(Fs['Ez'], A.mesh)

Now the PML one

In [ ]:
k_low = B.ks[0].real * B.scale

plotlogf(B.determinant, .999*k_low, 1.001 * k_low, -.015,.015, nu, 'pcb',pml,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(.9999*k_low)
imag = 0
x0 = np.array(guess + imag*1j)

beta = newton(B.determinant, x0, args=(nu, 'pcb', pml), tol = 1e-17)
beta

In [ ]:
Fs = B.all_fields(beta, nu, 'pcb', pml)

In [ ]:
Draw(Fs['Ez'], B.mesh)

# Regular Wavelength Loss Study


In [ ]:
d = 4.0775e-05
ts = [d, .5*d]
ns = [lambda x: n_air, lambda x: n_glass]
mats = ['air', 'glass']
maxhs = [.060, .010]
scale = 15e-6

betas_reg = np.zeros_like(wls, dtype=complex)
outer = 'h2'

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[2,2], wl=wls[i], ns=ns, mats=mats, scale=scale, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas_reg[i] = beta
    

In [ ]:
%matplotlib notebook
plt.figure(figsize=(8,5))
plt.plot(wls, -betas_reg.imag, color='green', linewidth=.9)
plt.yscale('log')


In [ ]:
# np.save('../N2_varying_thickness_studies/data/N0_betas', betas1)

# Exact PML Wavelength Loss Study


In [ ]:
ts = [4.0775e-05, 1e-5, 1.5e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]

mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [6, 33, 44]
scale = 15e-6

pml = {'alpha':.028, 'R0':rhos[-2]}

betas_pml = np.zeros_like(wls, dtype=complex)
outer = 'pcb'

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    B = BraggExact(ts=ts, maxhs=maxhs, wl=wls[i], ns=ns, mats=mats, scale=scale, no_mesh=True)

    k_low = B.ks[0] * B.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(B.determinant, x0, args=(nu, outer, pml), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(B.determinant(beta, nu, outer, pml)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas_pml[i] = beta
    

Load cluster results

In [ ]:
cluster_betas = np.load('/home/pv/local/convergence/bragg_fiber/wavelength/N0/clean_betas_im.npy')
# exact_scaled_betas = np.load('/home/pv/local/convergence/bragg_fiber/wavelength/N0/exact_scaled_betas.npy')

In [ ]:
%matplotlib inline
plt.figure(figsize=(15,9))
plt.title('\nN0 Configuration Spectral Loss Comparison:\nExact without PML vs\nExact with PML and perfectly conducting BCs vs \nNumerical\n',
         fontsize=18)
plt.plot(wls, -betas_reg.imag, linewidth=1.5, linestyle=(0,(8,6)), label='exact')
plt.plot(wls, -betas_pml.imag, linewidth=2, label='exact_PML')
plt.ylabel('CL', fontsize=16)
plt.xlabel('\n$\lambda$', fontsize=16)
plt.plot(wls, cluster_betas * A.scale, linewidth=1.5, label='cluster')
# plt.yscale('log')
plt.legend(fontsize=14);

# Repeat for several alpha values for PML

In [ ]:
ts = [4.0775e-05, 1e-5, 1.5e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]

mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [6, 33, 44]
scale = 15e-6

pml = {'alpha':.01, 'R0':rhos[-2]}

betas_pml2 = np.zeros_like(wls, dtype=complex)
outer = 'pcb'

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    B = BraggExact(ts=ts, maxhs=maxhs, wl=wls[i], ns=ns, mats=mats, scale=scale, no_mesh=True)

    k_low = B.ks[0] * B.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(B.determinant, x0, args=(nu, outer, pml), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(B.determinant(beta, nu, outer, pml)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas_pml2[i] = beta
    

In [ ]:
ts = [4.0775e-05, 1e-5, 1.5e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]

mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [6, 33, 44]
scale = 15e-6

pml = {'alpha':.02, 'R0':rhos[-2]}

betas_pml3 = np.zeros_like(wls, dtype=complex)
outer = 'pcb'

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    B = BraggExact(ts=ts, maxhs=maxhs, wl=wls[i], ns=ns, mats=mats, scale=scale, no_mesh=True)

    k_low = B.ks[0] * B.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(B.determinant, x0, args=(nu, outer, pml), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(B.determinant(beta, nu, outer, pml)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas_pml3[i] = beta
    

In [ ]:
ts = [4.0775e-05, 1e-5, 1.5e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]

mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [6, 33, 44]
scale = 15e-6

pml = {'alpha':.03, 'R0':rhos[-2]}

betas_pml4 = np.zeros_like(wls, dtype=complex)
outer = 'pcb'

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    B = BraggExact(ts=ts, maxhs=maxhs, wl=wls[i], ns=ns, mats=mats, scale=scale, no_mesh=True)

    k_low = B.ks[0] * B.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(B.determinant, x0, args=(nu, outer, pml), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(B.determinant(beta, nu, outer, pml)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas_pml4[i] = beta
    

In [ ]:
ts = [4.0775e-05, 1e-5, 1.5e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]

mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [6, 33, 44]
scale = 15e-6

pml = {'alpha':.04, 'R0':rhos[-2]}

betas_pml5 = np.zeros_like(wls, dtype=complex)
outer = 'pcb'

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    B = BraggExact(ts=ts, maxhs=maxhs, wl=wls[i], ns=ns, mats=mats, scale=scale, no_mesh=True)

    k_low = B.ks[0] * B.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(B.determinant, x0, args=(nu, outer, pml), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(B.determinant(beta, nu, outer, pml)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas_pml5[i] = beta
    

Load cluster results

In [ ]:
%matplotlib inline
plt.figure(figsize=(15,9))
plt.title('\nN0 Configuration Spectral Loss Comparison:\nExact without PML vs\nExact with PML and perfectly conducting BCs for several alphas\n',
         fontsize=18)
plt.plot(wls, -betas_reg.imag, linewidth=1.5, label='exact')
plt.plot(wls, -betas_pml2.imag, linewidth=1.5, label='alpha=.01')
plt.plot(wls, -betas_pml3.imag, linewidth=1.5, label='alpha=.02')
plt.plot(wls, -betas_pml4.imag, linewidth=1.5, label='alpha=.03')
plt.plot(wls, -betas_pml5.imag, linestyle=(0,(3,3)), linewidth=1.5, label='alpha=.04')

plt.ylabel('CL', fontsize=16)
plt.xlabel('\n$\lambda$', fontsize=16)

plt.yscale('log')
plt.legend(fontsize=14);

In [ ]:
ts = [4.0775e-05, 1e-5, 1.5e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]

mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [6, 33, 44]
scale = 15e-6

pml = {'alpha':.05, 'R0':rhos[-2]}

betas_pml6 = np.zeros_like(wls, dtype=complex)
outer = 'pcb'

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    B = BraggExact(ts=ts, maxhs=maxhs, wl=wls[i], ns=ns, mats=mats, scale=scale, no_mesh=True)

    k_low = B.ks[0] * B.scale
    guess = np.array(.99999 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(B.determinant, x0, args=(nu, outer, pml), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(B.determinant(beta, nu, outer, pml)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas_pml6[i] = beta
    

In [ ]:
%matplotlib inline
plt.figure(figsize=(15,9))
plt.title('\nN0 Configuration Spectral Loss Comparison:\nExact without PML vs\nExact with PML and perfectly conducting BCs for several alphas\n',
         fontsize=18)
plt.plot(wls, -betas_reg.imag, linewidth=2, label='exact')
# plt.plot(wls, -betas_pml2.imag, linewidth=1.5, label='alpha=.01')
# plt.plot(wls, -betas_pml3.imag, linewidth=1.5, label='alpha=.02')
# plt.plot(wls, -betas_pml4.imag, linewidth=1.5, label='alpha=.03')
plt.plot(wls, -betas_pml5.imag, linestyle=(0,(5,3)), linewidth=2, label='alpha=.04')
plt.plot(wls, -betas_pml6.imag, linestyle=(0,(5,3)), linewidth=2, label='alpha=.05')

plt.ylabel('CL', fontsize=16)
plt.xlabel('\n$\lambda$', fontsize=16)
plt.xlim(1.6e-6, 1.7e-6)
plt.ylim(.00012, .00014)
# plt.yscale('log')
plt.legend(fontsize=14);